In [ ]:
import joblib
import json
from sklearn.metrics import balanced_accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score

# Load the model
model = joblib.load(r"C:\Users\franc\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\HEK293\model_LGBM_173f.joblib")

# Load the external data
with open(r"C:\Users\franc\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\HEK293\curated_data\x\binary\x_ext\x_ext_ecfp_fp_2_1024.json", 'r') as f:
    x = json.load(f)

with open(r"C:\Users\franc\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\HEK293\curated_data\y\binary\y_ext\y_ext_ecfp_fp_2_1024.json", 'r') as f:
    y = json.load(f)


In [ ]:
import random
random.shuffle(y)

In [ ]:
import numpy as np
import pandas as pd

X_infos_df = pd.DataFrame(x)
X_infos_df = X_infos_df.add_prefix('var_')

features_sel = np.array(['var_1', 'var_2', 'var_4', 'var_6', 'var_15', 'var_19', 'var_23',
       'var_29', 'var_33', 'var_36', 'var_41', 'var_42', 'var_45',
       'var_46', 'var_63', 'var_64', 'var_72', 'var_74', 'var_80',
       'var_83', 'var_86', 'var_90', 'var_92', 'var_114', 'var_115',
       'var_117', 'var_119', 'var_121', 'var_128', 'var_136', 'var_138',
       'var_139', 'var_140', 'var_175', 'var_191', 'var_197', 'var_209',
       'var_212', 'var_216', 'var_218', 'var_222', 'var_233', 'var_242',
       'var_249', 'var_250', 'var_255', 'var_271', 'var_283', 'var_284',
       'var_288', 'var_294', 'var_301', 'var_310', 'var_314', 'var_316',
       'var_322', 'var_333', 'var_352', 'var_361', 'var_367', 'var_378',
       'var_381', 'var_383', 'var_387', 'var_389', 'var_393', 'var_407',
       'var_418', 'var_428', 'var_452', 'var_456', 'var_458', 'var_472',
       'var_480', 'var_481', 'var_482', 'var_484', 'var_486', 'var_491',
       'var_492', 'var_511', 'var_518', 'var_519', 'var_521', 'var_523',
       'var_531', 'var_540', 'var_549', 'var_551', 'var_556', 'var_561',
       'var_562', 'var_573', 'var_574', 'var_579', 'var_585', 'var_590',
       'var_602', 'var_628', 'var_641', 'var_646', 'var_650', 'var_651',
       'var_652', 'var_656', 'var_659', 'var_665', 'var_667', 'var_669',
       'var_673', 'var_675', 'var_689', 'var_692', 'var_695', 'var_698',
       'var_699', 'var_703', 'var_713', 'var_714', 'var_715', 'var_718',
       'var_726', 'var_730', 'var_736', 'var_738', 'var_739', 'var_741',
       'var_751', 'var_758', 'var_767', 'var_772', 'var_778', 'var_785',
       'var_792', 'var_798', 'var_799', 'var_801', 'var_802', 'var_807',
       'var_828', 'var_831', 'var_832', 'var_841', 'var_842', 'var_843',
       'var_847', 'var_849', 'var_852', 'var_864', 'var_875', 'var_881',
       'var_883', 'var_886', 'var_890', 'var_893', 'var_896', 'var_899',
       'var_904', 'var_922', 'var_926', 'var_932', 'var_935', 'var_940',
       'var_946', 'var_950', 'var_976', 'var_980', 'var_985', 'var_998',
       'var_1005', 'var_1008', 'var_1017', 'var_1019'], dtype=object)


In [ ]:
#features = sorted(features_sel, reverse = False, key = lambda ele: int(ele.split('_')[1]))
features = sorted(features_sel, reverse = False, key = lambda ele: int(ele.split('_')[1]))
X_new = X_infos_df[features]
X_new 

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

# Perform external validation
preds = model.predict(X_new)

# Calculate metrics
balanced_accuracy = round(balanced_accuracy_score(y, preds), 2)
f1 = round(f1_score(y, preds, average='weighted'), 2)
roc_auc = round(roc_auc_score(y, preds, multi_class='ovr'), 2)
mcc = round(matthews_corrcoef(y, preds), 2)
precision = round(precision_score(y, preds, average='weighted'), 2)

# Calculate confusion matrix
cm = confusion_matrix(y, preds)

# Calculate sensitivity and specificity
sensitivity = round(cm[1, 1] / (cm[1, 1] + cm[1, 0]), 2)  # Corrected
specificity = round(cm[0, 0] / (cm[0, 0] + cm[0, 1]), 2)  # Unchanged

# Print metrics
print(f"Balanced Accuracy: {balanced_accuracy}")
print(f"F1-score: {f1}")
print(f"AUC: {roc_auc}")
print(f"MCC: {mcc}")
print(f"Precision: {precision}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

# Print confusion matrix
print("\nConfusion Matrix:")
# Assign TP, FP, TN, FN
TP = cm[1, 1]
FP = cm[0, 1]
TN = cm[0, 0]
FN = cm[1, 0]

# Print TP, FP, TN, FN
print(f"True Positives: {TP}")
print(f"False Positives: {FP}")
print(f"True Negatives: {TN}")
print(f"False Negatives: {FN}")